In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('output_responses.csv')
df.iloc[:, -1][0]

'["I\'m sorry to hear that you\'re feeling down. It\'s important to take care of your mental well-being. Have you considered attending some of the workshops or groups offered by CAPS at UCSD? They can provide support and strategies to help you navigate your feelings. You can check out the Tritons RISE Workshops or consider signing up for a Let\'s Talk session for some informal support. Remember, you can also access free tools like Headspace for mindfulness and relaxation. If you\'re interested in joining a psychotherapy group or need professional counseling, please contact the CAPS Central Office at (858) 534-3755 to arrange an appointment. You\'re not alone, and there are resources available to help you through this!", \'I\\\'m sorry to hear that you\\\'re feeling down lately. It\\\'s important to take care of your mental health. Have you considered attending one of CAPS\\\' Tritons RISE Workshops? These workshops are designed to help promote your peak performance in various areas of 